# Neo4j MCP Server HTTP Connection Setup

This notebook demonstrates how to create a Databricks HTTP connection to the Neo4j MCP server deployed on Azure Container Apps. Once configured, you can query Neo4j graph data directly from SQL using the `http_request` function.

## What This Notebook Does

1. Validates that secrets are configured in Databricks
2. Creates an HTTP connection in Unity Catalog with bearer token authentication
3. Tests the connection by calling MCP tools (get-schema, read-cypher)
4. Demonstrates how to parse and use the results

## Prerequisites

- **Databricks Runtime**: 15.4 LTS or later, or SQL warehouse 2023.40+
- **Unity Catalog**: Must be enabled on your workspace
- **Secrets configured**: Run `scripts/setup_databricks_secrets.sh` before this notebook
- **MCP server deployed**: The Neo4j MCP server must be running on Azure Container Apps

## Security Note

This integration provides **READ-ONLY** access to Neo4j. The `write-cypher` tool is intentionally excluded to prevent accidental data modifications from analytics workflows.

## Configuration

Update these values to match your environment. The secret scope should match what you used when running `setup_databricks_secrets.sh`.

In [ ]:
# Configuration - update these values for your environment
SECRET_SCOPE = "mcp-neo4j-secrets"           # Must match the scope used in setup_databricks_secrets.sh
CONNECTION_NAME = "neo4j_azure_beta_mcp"     # Name for the HTTP connection in Unity Catalog
CATALOG = "mcp_demo_catalog"                  # Unity Catalog catalog name

## Step 1: Validate Secrets

First, verify that the required secrets exist in Databricks. If this step fails, run `scripts/setup_databricks_secrets.sh` from your local machine.

In [ ]:
# Validate that secrets are configured
try:
    endpoint = dbutils.secrets.get(scope=SECRET_SCOPE, key="endpoint")
    api_key = dbutils.secrets.get(scope=SECRET_SCOPE, key="api_key")
    mcp_path = dbutils.secrets.get(scope=SECRET_SCOPE, key="mcp_path")
    
    print(f"Secrets validated successfully!")
    print(f"  Endpoint: {endpoint}")
    print(f"  MCP Path: {mcp_path}")
    print(f"  API Key: [REDACTED - {len(api_key)} characters]")
except Exception as e:
    print(f"ERROR: Failed to retrieve secrets from scope '{SECRET_SCOPE}'")
    print(f"Error: {e}")
    print("")
    print("To fix this, run the setup script from your local machine:")
    print(f"  ./scripts/setup_databricks_secrets.sh {SECRET_SCOPE}")
    raise

## Step 2: Create the HTTP Connection

Create an HTTP connection in Unity Catalog. This connection:
- Points to the MCP server endpoint
- Uses bearer token authentication with the API key from secrets
- Can be shared with other users via Unity Catalog permissions

**Implementation Note**: We use `spark.sql()` with Python f-strings instead of `%%sql` magic because the `CREATE CONNECTION` OPTIONS clause requires constant expressions. Python string interpolation allows us to dynamically insert the host, path, and secret scope values.

**MCP Integration Note**: For full Databricks AI/MCP integration features, you may need to manually enable the "Is mcp connection" checkbox in the Catalog Explorer UI after creating the connection. Navigate to: **Catalog > External Data > Connections > neo4j_mcp > Edit** and check the MCP option. This setting is not currently available via SQL DDL.

**Note**: If the connection already exists, you'll see an error. Use the cleanup cell at the bottom to drop it first, then re-run this cell.

In [ ]:
# Get connection parameters from secrets
endpoint = dbutils.secrets.get(scope=SECRET_SCOPE, key="endpoint")
mcp_path = dbutils.secrets.get(scope=SECRET_SCOPE, key="mcp_path")

# Use full endpoint URL including https:// as the host
# (Databricks HTTP connections expect the full URL with protocol)
host = endpoint if endpoint.startswith("https://") else f"https://{endpoint}"

print(f"Creating HTTP connection '{CONNECTION_NAME}'...")
print(f"  Host: {host}")
print(f"  Port: 443")
print(f"  Base Path: {mcp_path}")
print(f"  Secret Scope: {SECRET_SCOPE}")

In [ ]:
# Create the HTTP connection with bearer token authentication
# The bearer_token references the secret, so the actual key is never exposed
# Note: Using spark.sql() because OPTIONS requires constant expressions (no variable interpolation in %%sql)

# Use catalog-qualified name for the connection
full_connection_name = f"{CATALOG}.{CONNECTION_NAME}"

create_connection_sql = f"""
CREATE CONNECTION IF NOT EXISTS {full_connection_name} TYPE HTTP
OPTIONS (
  host '{host}',
  port '443',
  base_path '{mcp_path}',
  bearer_token secret ('{SECRET_SCOPE}', 'api_key')
)
"""

print("Executing SQL:")
print(create_connection_sql)

spark.sql(create_connection_sql)
print(f"\nConnection created: {full_connection_name}")

In [ ]:
# Verify connection was created
print(f"Connection '{full_connection_name}' created successfully!")
print("")
print(f"You can now use this connection with http_request() in SQL.")
print(f"")
print(f"To use in Databricks Playground:")
print(f"  1. Go to Add tools > MCP Servers tab")
print(f"  2. Under 'External MCP Servers', select '{full_connection_name}' from the Unity Catalog Connection dropdown")

## Step 3: Test the Connection - List Tools

The MCP protocol uses JSON-RPC. Let's first list the available tools to verify the connection works.

In [ ]:
import json

# MCP JSON-RPC request to list tools
list_tools_request = json.dumps({
    "jsonrpc": "2.0",
    "method": "tools/list",
    "id": 1
})

print("Request payload:")
print(list_tools_request)

In [ ]:
# List available MCP tools
# Note: Using Python to reference the full_connection_name variable
list_tools_sql = f"""
SELECT http_request(
  conn => '{full_connection_name}',
  method => 'POST',
  path => '',
  headers => map('Content-Type', 'application/json'),
  json => '{{"jsonrpc":"2.0","method":"tools/list","id":1}}'
) AS response
"""

result = spark.sql(list_tools_sql)
display(result)

## Step 4: Get Neo4j Schema

Call the `get-schema` tool to retrieve the Neo4j database schema, including node labels, relationship types, and properties.

In [ ]:
# Get the Neo4j database schema
get_schema_sql = f"""
SELECT http_request(
  conn => '{full_connection_name}',
  method => 'POST',
  path => '',
  headers => map('Content-Type', 'application/json'),
  json => '{{"jsonrpc":"2.0","method":"tools/call","params":{{"name":"get-schema","arguments":{{}}}},"id":2}}'
) AS response
"""

result = spark.sql(get_schema_sql)
display(result)

## Step 5: Execute a Read Query

Call the `read-cypher` tool to execute a read-only Cypher query against Neo4j.

**Important**: Only read queries are permitted through this connection. The `write-cypher` tool is intentionally not exposed.

In [ ]:
import json

# Example: Count nodes by label
cypher_query = "MATCH (n) RETURN labels(n) AS label, count(*) AS count ORDER BY count DESC LIMIT 10"

# Build the MCP request
read_cypher_request = json.dumps({
    "jsonrpc": "2.0",
    "method": "tools/call",
    "params": {
        "name": "read-cypher",
        "arguments": {
            "query": cypher_query
        }
    },
    "id": 3
})

print("Cypher query:")
print(cypher_query)
print("")
print("MCP request:")
print(read_cypher_request)

In [ ]:
# Store the request for use in SQL
spark.conf.set("cypher_request", read_cypher_request)

In [ ]:
# Execute a read-only Cypher query
# Escape curly braces for f-string and single quotes for SQL
escaped_request = read_cypher_request.replace("'", "''")

cypher_sql = f"""
SELECT http_request(
  conn => '{full_connection_name}',
  method => 'POST',
  path => '',
  headers => map('Content-Type', 'application/json'),
  json => '{escaped_request}'
) AS response
"""

result = spark.sql(cypher_sql)
display(result)

## Step 6: Parse and Display Results

The MCP response is JSON. Let's parse it and display the results in a more readable format.

In [ ]:
from pyspark.sql.functions import col, from_json, get_json_object
from pyspark.sql.types import StringType, StructType, StructField, ArrayType

# Execute the query and get the response
escaped_request = read_cypher_request.replace("'", "''")

result_df = spark.sql(f"""
    SELECT http_request(
      conn => '{full_connection_name}',
      method => 'POST',
      path => '',
      headers => map('Content-Type', 'application/json'),
      json => '{escaped_request}'
    ) AS response
""")

# Extract the response body
response_row = result_df.first()
if response_row:
    response = response_row["response"]
    print("Raw response:")
    print(json.dumps(json.loads(response["text"]), indent=2) if "text" in response else response)

## Helper Function: Query Neo4j

Here's a reusable function to query Neo4j through the MCP connection.

In [ ]:
import json

def query_neo4j(cypher_query: str, connection_name: str = None) -> dict:
    """
    Execute a read-only Cypher query against Neo4j via the MCP HTTP connection.
    
    Args:
        cypher_query: The Cypher query to execute (read-only)
        connection_name: Full name of the HTTP connection (default: uses full_connection_name from notebook)
    
    Returns:
        dict: The parsed JSON response from Neo4j
    """
    if connection_name is None:
        connection_name = full_connection_name
    
    # Build the MCP request
    request_payload = json.dumps({
        "jsonrpc": "2.0",
        "method": "tools/call",
        "params": {
            "name": "read-cypher",
            "arguments": {
                "query": cypher_query
            }
        },
        "id": 1
    })
    
    # Escape single quotes for SQL
    escaped_payload = request_payload.replace("'", "''")
    
    # Execute the query
    result_df = spark.sql(f"""
        SELECT http_request(
          conn => '{connection_name}',
          method => 'POST',
          path => '',
          headers => map('Content-Type', 'application/json'),
          json => '{escaped_payload}'
        ) AS response
    """)
    
    # Parse the response
    response_row = result_df.first()
    if response_row and "response" in response_row.asDict():
        response = response_row["response"]
        if "text" in response:
            return json.loads(response["text"])
    return None

print("Helper function 'query_neo4j' is now available.")
print("")
print("Example usage:")
print(f'  result = query_neo4j("MATCH (n) RETURN count(n) AS total")')
print(f"  # Uses connection: {full_connection_name}")

In [ ]:
# Example: Use the helper function
result = query_neo4j("MATCH (n) RETURN count(n) AS total")
print("Query result:")
print(json.dumps(result, indent=2))

## Troubleshooting

### Common Issues

**1. Secret not found**
```
Secret does not exist with scope: mcp-neo4j-secrets and key: api_key
```
Solution: Run `./scripts/setup_databricks_secrets.sh` from your local machine.

**2. Connection already exists**
```
Connection 'neo4j_mcp' already exists
```
Solution: Use the cleanup cell below to drop the connection, then re-create it.

**3. HTTP request timeout**
```
Connection timed out
```
Solution: Verify the MCP server is running. Check the endpoint URL in your secrets.

**4. Authentication failed**
```
401 Unauthorized
```
Solution: The API key may be incorrect. Re-run `setup_databricks_secrets.sh` to refresh the secrets from `MCP_ACCESS.json`.

## Cleanup

Use this cell to drop the HTTP connection if you need to recreate it or clean up resources.

In [ ]:
# Uncomment the lines below to drop the connection
# spark.sql(f"DROP CONNECTION IF EXISTS {full_connection_name}")
# print(f"Connection '{full_connection_name}' dropped.")

## Next Steps

Now that you have a working HTTP connection to Neo4j:

1. **Explore the schema**: Use `get-schema` to understand your graph structure
2. **Write analytics queries**: Use `read-cypher` to query graph data for your analytics
3. **Join with Delta tables**: Combine graph query results with your Delta Lake data
4. **Share the connection**: Grant `USE CONNECTION` to other users via Unity Catalog

For more information:
- [Databricks HTTP Connections](https://docs.databricks.com/aws/en/query-federation/http)
- [Neo4j Cypher Query Language](https://neo4j.com/docs/cypher-manual/current/)
- [Model Context Protocol (MCP)](https://modelcontextprotocol.io/)